In [15]:
import pandas as pd
import time
import gc
import sklearn
from sklearn.impute import SimpleImputer
from xgboost import XGBClassifier
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_auc_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import mutual_info_classif
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from scipy.sparse import hstack
from scipy.sparse import coo_matrix
import xgboost as xgb
from scipy.stats import uniform
from sklearn.model_selection import ParameterSampler
from sklearn.metrics import balanced_accuracy_score, roc_auc_score, make_scorer
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import LabelEncoder
from scipy.stats import uniform
import ast
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from xgboost.callback import EarlyStopping
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

random_state = 42
np.random.seed(random_state)

In [3]:
ctr_15 = pd.read_csv('/Users/ionikullock/Desktop/UTDT-Tecnología Digital/TD VI/Trabajo práctico 2/Datos/ctr_15.csv')
ctr_16 = pd.read_csv('/Users/ionikullock/Desktop/UTDT-Tecnología Digital/TD VI/Trabajo práctico 2/Datos/ctr_16.csv')
ctr_17 = pd.read_csv('/Users/ionikullock/Desktop/UTDT-Tecnología Digital/TD VI/Trabajo práctico 2/Datos/ctr_17.csv') 
ctr_18 = pd.read_csv('/Users/ionikullock/Desktop/UTDT-Tecnología Digital/TD VI/Trabajo práctico 2/Datos/ctr_18.csv')  
data = pd.concat(([ctr_15,ctr_16,ctr_17,ctr_18]))
print("Tamaño del conjunto ", data.shape)
ctr_test = pd.read_csv('/Users/ionikullock/Desktop/UTDT-Tecnología Digital/TD VI/Trabajo práctico 2/Datos/ctr_test.csv')
print("Tamaño del conjunto ", ctr_test.shape)

Tamaño del conjunto  (4439125, 52)
Tamaño del conjunto  (1139639, 52)


In [4]:
# Lista de columnas a eliminar
columns_to_remove = [
    'device_id', 'action_list_1', 'action_list_2', 'action_categorical_0', 
    'auction_list_0', 'action_categorical_2', 'action_categorical_4', 
    'auction_categorical_0', 'auction_categorical_11', 'auction_categorical_3', 
    'creative_categorical_0', 'auction_categorical_7', 'Auction_time', 'auction_categorical_10', 'action_categorical_7', 'action_categorical_6',
    'auction_boolean_1', 'auction_boolean_2', 'creative_categorical_10',
    'auction_categorical_4', 'gender', 'creative_categorical_1', 
    'auction_categorical_6', 'auction_boolean_0', 'creative_categorical_3',
    'auction_categorical_2', 'action_categorical_5', 'action_list_0',
    'auction_categorical_1', 'auction_categorical_5', 'device_id_type',
    'auction_categorical_8', 'action_categorical_0', 'auction_categorical_0',
    'creative_categorical_9', 'creative_categorical_11', 'creative_categorical_5',
    'creative_categorical_7', 'creative_categorical_12', 'creative_categorical_8',
    'creative_categorical_4', 'creative_categorical_6', 'action_categorical_1',
    'creative_categorical_2', 'action_categorical_3', 'action_categorical_2',
    'auction_categorical_11', 'action_categorical_4',
    'creative_categorical_0', 'device_id'
]

data = data.drop(columns=columns_to_remove, errors='ignore')
ctr_test = ctr_test.drop(columns=columns_to_remove, errors='ignore')

id_col_test = ctr_test["id"]
ctr_test = ctr_test.drop(columns="id", errors='ignore')

print(f"Columnas eliminadas: {columns_to_remove}")
print(f"Tamaño del data después de la eliminación: {data.shape}")
print(f"Tamaño del test después de la eliminación: {ctr_test.shape}")


Columnas eliminadas: ['device_id', 'action_list_1', 'action_list_2', 'action_categorical_0', 'auction_list_0', 'action_categorical_2', 'action_categorical_4', 'auction_categorical_0', 'auction_categorical_11', 'auction_categorical_3', 'creative_categorical_0', 'auction_categorical_7', 'Auction_time', 'auction_categorical_10', 'action_categorical_7', 'action_categorical_6', 'auction_boolean_1', 'auction_boolean_2', 'creative_categorical_10', 'auction_categorical_4', 'gender', 'creative_categorical_1', 'auction_categorical_6', 'auction_boolean_0', 'creative_categorical_3', 'auction_categorical_2', 'action_categorical_5', 'action_list_0', 'auction_categorical_1', 'auction_categorical_5', 'device_id_type', 'auction_categorical_8', 'action_categorical_0', 'auction_categorical_0', 'creative_categorical_9', 'creative_categorical_11', 'creative_categorical_5', 'creative_categorical_7', 'creative_categorical_12', 'creative_categorical_8', 'creative_categorical_4', 'creative_categorical_6', 'act

In [5]:
def group_rare_categories(column, threshold):
    freq = column.value_counts(normalize=True)
    rare_categories = freq[freq < threshold].index
    return column.apply(lambda x: 'Other' if x in rare_categories else x)

threshold = 0.05

categorical_columns = data.select_dtypes(include=['object']).columns.tolist()
categorical_columns_eval = ctr_test.select_dtypes(include=['object']).columns.tolist()

#categorical_columns = ['auction_categorical_12', 'auction_categorical_3', 'auction_categorical_7', 'auction_categorical_9']
#list_columns = ['action_list_2', 'action_list_1', 'auction_list_0']

for col in categorical_columns:
    data[col] = group_rare_categories(data[col], threshold)

for col in categorical_columns_eval:
    ctr_test[col] = group_rare_categories(ctr_test[col], threshold)
pd_ohe = pd.get_dummies(data,
                        columns=categorical_columns + ['has_video'],  # Incluye has_video también
                        sparse=False,    # Devolver una matriz no rala para facilitar la alineación
                        dummy_na=False,  # No agregar columna para NaNs.
                        dtype=int        # XGBoost no trabaja con 'object'; necesitamos que sean numéricos.
                       )

pd_ohe_eval = pd.get_dummies(ctr_test,
                             columns=categorical_columns_eval + ['has_video'],  # Incluye has_video también
                             sparse=False,    # Devolver una matriz no rala
                             dummy_na=False,  # No agregar columna para NaNs.
                             dtype=int        # XGBoost no trabaja con 'object'; necesitamos que sean numéricos.
                            )

pd_ohe_eval = pd_ohe_eval.reindex(columns=pd_ohe.columns, fill_value=0)

pd_ohe_eval['id'] = id_col_test.values

print("Shape de data_drop después de OneHotEncoding:", pd_ohe.shape)
print("Shape de eval_data_drop después de OneHotEncoding:", pd_ohe_eval.shape)

pd_ohe.info()
pd_ohe_eval.info()

Shape de data_drop después de OneHotEncoding: (4439125, 16)
Shape de eval_data_drop después de OneHotEncoding: (1139639, 17)
<class 'pandas.core.frame.DataFrame'>
Index: 4439125 entries, 0 to 1013580
Data columns (total 16 columns):
 #   Column                           Dtype  
---  ------                           -----  
 0   Label                            int64  
 1   auction_age                      float64
 2   auction_bidfloor                 float64
 3   auction_time                     float64
 4   creative_height                  float64
 5   creative_width                   float64
 6   timezone_offset                  float64
 7   auction_categorical_12_9c3324c7  int64  
 8   auction_categorical_12_Other     int64  
 9   auction_categorical_12_b4e520dd  int64  
 10  auction_categorical_12_e9e0850e  int64  
 11  auction_categorical_9_11105871   int64  
 12  auction_categorical_9_Other      int64  
 13  auction_categorical_9_d4eea9a2   int64  
 14  has_video_False           

In [6]:
y = pd_ohe[['Label']].copy() # Usamos copy para no modificar un view abajo, ya que genera un warning.
X = pd_ohe.drop('Label', axis = 1)

X_train, X_val, Y_train, Y_val = train_test_split(X, y,
                                                  train_size = 0.8,
                                                  random_state = 32,
                                                  stratify = y)

X_test = pd_ohe_eval.drop('Label', axis = 1)

print(X_test.shape)
print(f'Cantidad de datos de train: {len(X_train)}')
print(f'Cantidad de datos de validación: {len(X_val)}')


(1139639, 16)
Cantidad de datos de train: 3551300
Cantidad de datos de validación: 887825


In [17]:

# Define el modelo
clf_xgb = xgb.XGBClassifier(objective='binary:logistic',
                            seed=42,
                            eval_metric='auc',
                            use_label_encoder=False,
                            n_estimators=200)

# Usar el callback EarlyStopping para la parada temprana
clf_xgb.fit(X_train, Y_train, 
            verbose=True, 
            eval_set=[(X_val, Y_val)], 
            callbacks=[EarlyStopping(rounds=10, save_best=True)])  # Usar EarlyStopping callback

# Obtener la mejor iteración
best_iteration = clf_xgb.best_iteration

# Predecir probabilidades en el conjunto de validación con el mejor modelo
y_pred_proba = clf_xgb.predict_proba(X_val, ntree_limit=best_iteration)[:, 1]

# Calcular la curva ROC
fpr, tpr, thresholds = roc_curve(Y_val, y_pred_proba)
roc_auc = auc(fpr, tpr)

# Graficar la curva ROC
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'AUC ROC (area = {roc_auc:.4f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc="lower right")
plt.show()

TypeError: XGBClassifier.fit() got an unexpected keyword argument 'callbacks'

In [57]:
X_test['id'] = range(1, len(X_test) + 1)
y_pred_test = clf_xgb.predict_proba(X_test.drop(columns=["id"]))[:, clf_xgb.classes_ == 1].squeeze()
submission_df = pd.DataFrame({"id": X_test["id"], "Label": y_pred_test})
submission_df["id"] = submission_df["id"].astype(int)
submission_df.to_csv("XGBOOTS.csv", sep=",", index=False)